In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as grid_spec
from matplotlib.colors import ListedColormap
import itertools
from datetime import datetime

import trueskill
from scipy.stats import norm
from sqlalchemy import select, insert, update, create_engine, Date, and_
from sqlalchemy.orm import Session, aliased

import data
from report_utils import ( 
    plot_match_qualities, 
    plot_ridge_skill_distributions, 
    plot_paired_skill_distributions, 
    plot_player_positions_bar,
    win_probability
    )
from common_queries import (
    get_player_skill_for_team,
    leaderboard
)


In [ ]:
sns.set_context("notebook")
sns.set_style("white")

In [ ]:
def plot_teams_for_competiton(table_position, players):
    df_dict = [
    {
        "name" : player.name,
        "rating" : player.rating,
        "club" : player.club,
        "club_rank" : table_position.index(player.club)
    }
    for player in players
]

    df = pd.DataFrame(df_dict)
    df.sort_values(by=["club", "rating"], inplace=True)
    df.reset_index(inplace=True)

    fig, ax = plt.subplots(1,1, figsize=(20,14))
    ax = sns.scatterplot(data=df, x="rating", y="club_rank", hue="club")
    ax.set_yticks(range(len(table_position)), labels=table_position)
    ax.set_ylabel("Club nach Tabellenplatz")
    ax.legend(loc='upper right')
    elevate = 0.1

    for i, player in df.iterrows():
        if i < (len(df) - 1):
            if abs(player.rating - df.iloc[i+1]["rating"]) < 4: 
                elevate = next_elevation(elevate)
            else:
                elevate = -0.1 if elevate == 0.1 else 0.1
        ax.text(player.rating, player.club_rank + elevate, str(player["name"]), horizontalalignment="left", fontsize="xx-small", mouseover=True)
        
    return fig

def next_elevation(old):
    if old == 0.2:
        return -0.2
    elif old == -0.1:
        return 0.1
    
    return old+0.1


In [ ]:
# TODO Better layout for this notebook
# Functions in notebook for:
#   - Previous lineup
#   - Position distribution per player
#   - Get best possible lineup (with best 4 of them?) 
engine = create_engine("sqlite:///../darts-json.db")
season = datetime(2022, 8, 1)
conservative=False



### A Team

In [ ]:
a_team = "Landesliga 1"

table_position = [
    "Hannover 96",
    "DC Dartskulls Basche e.V.",
    "SDC Osnabrück",
    "DC No Mercy Gronau e.V.",
    "Sieben Zwerge Dart Team e.V.",
    "Neustädter Rübendarter e.V.",
    "Flying Bulldogs",
    "TuS Holzhausen/Porta e.V.",
]

players = leaderboard(engine, a_team, season=season, conservative=conservative)

_ = plot_teams_for_competiton(table_position, players)
plt.savefig(f"../img/a_team_{season.year}_{conservative=}.png")

### B Team

In [ ]:
b_team = "Verbandsliga 1"

table_position = [
    "GVO Oldenburg",
    "Hannover 96",
    "SuS Buer",
    "SC Drop Out Seelze e.V.",
    "DC Hameln 79 e.V.",
    "DC Dudensen",
    "DC No Mercy Gronau e.V.",
    "Ith Bulls Lauenstein",
]

players = leaderboard(engine, b_team, season=season, conservative=conservative)

_ = plot_teams_for_competiton(table_position, players)
plt.savefig(f"../img/b_team_{season.year}_{conservative=}.png")

### C Team

In [ ]:
c_team = "DBH Bezirksoberliga"

table_position = [
    "1.Dartclub Hildesheim e.V.",
    "DC Alpakas Hille e.V.",
    "Hannover 96",
    "DC Dartskulls Basche e.V.",
    "Steelbreakers Lehrte e.V.",
    "Riverside Sharks Hehlen",
    "DC Langendamm e.V.",
    "VFL Bad Nenndorf",
]

players = leaderboard(engine, c_team, season=season, conservative=conservative)

_ = plot_teams_for_competiton(table_position, players)
plt.savefig(f"../img/c_team_{season.year}_{conservative=}.png")

### D Team

In [ ]:
d_team = "DBH Bezirksliga 2"

table_position = [
    "DC Hameln 79 e.V.",
    "PDC Hannover e.V.",
    "Sieben Zwerge Dart Team e.V.",
    "1.Dartclub Hildesheim e.V.",
    "Shakespeare Dartists Hannover e.V.",
    "Hannover 96",
    "Flying Owls Hörsum e.V.",
    "Ith Lions Coppenbrügge",
    "DC Vikings Oesselse/Gleidingen",
    "Die Zeitdiebe Herrenhausen e.V."
]

players = leaderboard(engine, d_team, season=season, conservative=conservative)

_ = plot_teams_for_competiton(table_position, players)
plt.savefig(f"../img/d_team_{season.year}_{conservative=}.png")

### E-Team

In [ ]:
e_team = "DBH Bezirksklasse 4"

table_position = [
    "Dart Devils Glissen",
    "DC Sloths Steyerberg",
    "DSV Stingrays Hannover e.V.",
    "TuS Lühnde e.V.",
    "Hannover 96",
    "Hannoverscher DSC",
    "SV Northen Lenthe e.V.",
    "TSV Egestorf e.V.",
    "Shakespeare Dartists Hannover e.V.",
    "DC Wild Rovers"
]

players = leaderboard(engine, e_team, season=season, conservative=conservative)

_ = plot_teams_for_competiton(table_position, players)
plt.savefig(f"../img/e_team_{season.year}_{conservative=}.png")

### F-Team

In [ ]:
f_team = "DBH Kreisliga 9"

table_position = [
    "TuS Germania Hohnhorst",
    "FSG Pollhagen/Nordsehl/Lauenhagen",
    "DC Dartskulls Basche e.V.",
    "SV Victoria Sachsenhagen 1900 e.V.",
    "Mühlenberger SV e.V.",
    "SV Northen Lenthe e.V.",
    "Hannover 96",
    "SV Arminia Bunte Hunde e.V.",
    "Hannoverscher DSC",
    "VFL Bad Nenndorf"
]

players = leaderboard(engine, f_team, season=season, conservative=conservative)

_ = plot_teams_for_competiton(table_position, players)
plt.savefig(f"../img/f_team_{season.year}_{conservative=}.png")

### G-Team

In [ ]:
g_team = "DBH Kreisliga 2"

table_position = [
    "SC Victory e.V.",
    "DSV Stingrays Hannover e.V.",
    "SV Uetze e.V.",
    "Hannover 96",
    "SV Arminia Bunte Hunde e.V.",
    "Hannoverscher DSC",
    "Steelbreakers Lehrte e.V.",
    "Shakespeare Dartists Hannover e.V.",
    "DC Diabolo Hannover e.V.",
    "Die Zeitdiebe Herrenhausen e.V."
]

players = leaderboard(engine, g_team, season=season, conservative=conservative)

_ = plot_teams_for_competiton(table_position, players)
plt.savefig(f"../img/g_team_{season.year}_{conservative=}.png")
